In [1]:
# download data
!wget http://snap.stanford.edu/multiscale-interactome/data/data.tar.gz
!tar -xvf data.tar.gz

--2022-01-18 15:46:30--  http://snap.stanford.edu/multiscale-interactome/data/data.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 565295082 (539M) [application/x-gzip]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>] 539.11M  4.79MB/s    in 1m 51s  

2022-01-18 15:48:20 (4.88 MB/s) - ‘data.tar.gz’ saved [565295082/565295082]

data/
data/1_drug_to_protein.tsv
data/4_protein_to_biological_function.tsv
data/9_pharmgkb_df.tsv
data/5_biological_function_to_biological_function.tsv
data/8_bcm_df.tsv
data/6_drug_indication_df.tsv
data/10_top_msi.zip
data/._README.txt
data/README.txt
data/3_protein_to_protein.tsv
data/7_drug_classification_df.tsv
data/2_indication_to_protein.tsv


In [2]:
# clone the whole project so that we can use the pre-defined functions and classes
!git clone https://github.com/snap-stanford/multiscale-interactome

Cloning into 'multiscale-interactome'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 89 (delta 33), reused 81 (delta 28), pack-reused 0
Unpacking objects: 100% (89/89), done.


In [3]:
# move to the directory of multiscale-interactome so that we can work
%cd ./multiscale-interactome
# install reauired packages
%pip install -r requirements.txt

/content/multiscale-interactome
     |████████████████████████████████| 1.7 MB 31.3 MB/s 
     |████████████████████████████████| 17.3 MB 31 kB/s 
     |████████████████████████████████| 10.1 MB 65.6 MB/s 
     |████████████████████████████████| 227 kB 68.1 MB/s 
     |████████████████████████████████| 509 kB 48.4 MB/s 
     |████████████████████████████████| 25.2 MB 1.4 MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556008 sha256=b394841f5cbfa0b0b9a2c93364f869d0c49aa4914bbc1621a2abf45e3956a723
  Stored in directory: /root/.cache/pip/wheels/44/e6/b8/4efaab31158e9e9ca9ed80b11f6b11130bac9a9672b3cbbeaf
Successfully built networkx
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: pytz
    Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9
  Attempting uninstall: python-dateut

In [4]:
%load_ext autoreload
%autoreload 2

In [40]:
from msi.msi import MSI
from diff_prof.diffusion_profiles import DiffusionProfiles
import multiprocessing
import numpy as np
import pickle
import networkx as nx
import pandas as pd
import scipy
import os
import os.path as osp
from tests.msi import test_msi
from tests.diff_prof import test_diffusion_profiles

In [6]:
!pwd

/content/multiscale-interactome


In [7]:
#move data file to the path multiscale-interactome firstly, then run code
%mv /content/data ./
msi = MSI()
msi.load()

In [8]:
!unzip /content/multiscale-interactome/data/10_top_msi.zip


Archive:  /content/multiscale-interactome/data/10_top_msi.zip
   creating: 10_top_msi/
  inflating: 10_top_msi/DB00334_p_visit_array.npy  
  inflating: 10_top_msi/DB01068_p_visit_array.npy  
  inflating: 10_top_msi/DB00631_p_visit_array.npy  
  inflating: 10_top_msi/C0150055_p_visit_array.npy  
  inflating: 10_top_msi/C0002888_p_visit_array.npy  
  inflating: 10_top_msi/C0030305_p_visit_array.npy  
  inflating: 10_top_msi/C0042875_p_visit_array.npy  
  inflating: 10_top_msi/DB00069_p_visit_array.npy  
  inflating: 10_top_msi/DB00983_p_visit_array.npy  
  inflating: 10_top_msi/DB00389_p_visit_array.npy  
  inflating: 10_top_msi/epalrestat_p_visit_array.npy  
  inflating: 10_top_msi/DB09244_p_visit_array.npy  
  inflating: 10_top_msi/DB01186_p_visit_array.npy  
  inflating: 10_top_msi/DB01483_p_visit_array.npy  
  inflating: 10_top_msi/C0022116_p_visit_array.npy  
  inflating: 10_top_msi/DB11572_p_visit_array.npy  
  inflating: 10_top_msi/C0010674_p_visit_array.npy  
  inflating: 10_top_

In [ ]:
# useless
#test_msi()


In [9]:
# construct diffusion profiles & no need to change any of these parameters
dp = DiffusionProfiles(alpha = 0.8595436247434408, max_iter = 1000, tol = 1e-06, weights = {'down_biological_function': 4.4863053901688685, 'indication': 3.541889556309463, 'biological_function': 6.583155399238509, 'up_biological_function': 2.09685000906964, 'protein': 4.396695660380823, 'drug': 3.2071696595616364}, num_cores = 2, save_load_file_path = "results/")

dp.calculate_diffusion_profiles(msi)

In [10]:
# load saved diffusion profile
dp_saved = DiffusionProfiles(alpha = None, max_iter = None, tol = None, weights = None, num_cores = None, save_load_file_path = "results/")

msi.load_saved_node_idx_mapping_and_nodelist(dp_saved.save_load_file_path)

dp_saved.load_diffusion_profiles(msi.drugs_in_graph + msi.indications_in_graph)

In [11]:
#food molecule diffusion profile
dp_saved.drug_or_indication2diffusion_profile["FDB022665"]


array([2.25714581e-01, 2.94988044e-02, 2.89288595e-02, ...,
       3.48260736e-06, 2.21051011e-06, 2.21051011e-06])

In [12]:
#colorectal cancer diffusion profiles
dp_saved.drug_or_indication2diffusion_profile['RC']

array([2.24372634e-06, 3.25889607e-06, 1.24465017e-05, ...,
       1.18176097e-05, 4.41692151e-06, 4.41692151e-06])

In [21]:
# calculate the distance between two diffusion profiles
drug_to_protein = pd.read_csv('/content/multiscale-interactome/data/1_drug_to_protein.tsv', sep= '\t')
distance = {}
for fm in drug_to_protein['node_1']:
  array = dp_saved.drug_or_indication2diffusion_profile[fm]
  fm_distance = scipy.spatial.distance.correlation(dp_saved.drug_or_indication2diffusion_profile['RC'],array,w=None,centered=True)
  distance[fm] = fm_distance

In [51]:
distance['FDB012622']

0.7861166319260299

In [48]:
#Create dataframe
rows = []
fm_id_name = dict(zip(drug_to_protein['node_1'],drug_to_protein['node_1_name']))
indication = 'Colorectal Cancer'
for fm in distance.keys():
  fm_name = fm_id_name[fm]
  distance_value = distance[fm]
  rows.append([fm,fm_name,indication,distance_value])

outdir = osp.join('distance', 'colorectal cancer')
os.makedirs(outdir, exist_ok = True)
Colorectal_distance = pd.DataFrame(rows,columns=['Food molecule','Food name','Indication','Distance']).sort_values(by = ['Distance'])



In [46]:
!pip install openpyxl
import openpyxl

In [49]:
Colorectal_distance.to_excel(osp.join(outdir,'Colorectal_distance.xlsx'))

